In [ ]:
!pip install qiskit --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
from qiskit import QuantumCircuit
from qiskit.quantum_info import partial_trace, Statevector, entropy

In [ ]:


# --- 1. System Definition Parameters ---
N_QUBITS = 8  # Total number of degrees of freedom (qubits)
B_INITIAL_SIZE = N_QUBITS # Initial size of the Black Hole (B) subsystem
R_INITIAL_SIZE = 0        # Initial size of the Radiation (R) subsystem

# Import necessary libraries within this cell
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import partial_trace, Statevector, entropy


# --- 2. Evaporation Operation (Unitary) ---
def get_scrambling_unitary(b_qubits_indices, num_total_qubits):
    """
    Creates a more complex entangling unitary gate across the current black hole
    qubits to model entanglement and scrambling. Increased complexity.
    """
    qc = QuantumCircuit(num_total_qubits)
    # Apply multiple layers of CNOT gates and local rotations for more scrambling
    # Increased number of layers for potentially better chaotic evolution
    for _ in range(3): # Increased layers from 1 to 3
        for i in range(len(b_qubits_indices) - 1):
            qc.cx(b_qubits_indices[i], b_qubits_indices[i+1])
            # Add CNOTs in reverse order as well
            qc.cx(b_qubits_indices[i+1], b_qubits_indices[i])

        for qubit_index in b_qubits_indices:
            qc.rz(np.random.rand() * 2 * np.pi, qubit_index)
            qc.ry(np.random.rand() * 2 * np.pi, qubit_index)
            qc.rx(np.random.rand() * 2 * np.pi, qubit_index) # Add RX rotations

    # We return the whole circuit compositionally for simplicity in Colab
    return qc

def initialize_system(n_qubits):
    """Initializes the total system in a pure state (|00...0>)."""
    qc = QuantumCircuit(n_qubits)
    # All qubits start in the pure state |0>
    return qc

def simulate_evaporation(initial_qc, n_total):
    """
    Simulates the black hole evaporation process step-by-step and calculates
    the entanglement entropy of the radiation (R) subsystem.
    """
    current_circuit = initial_qc.copy()

    # Track results
    entanglement_entropies = []

    # Start with all qubits in B (Black Hole), R is empty
    b_qubits = list(range(n_total)) # Qubit indices currently in B
    r_qubits = []                    # Qubit indices currently in R

    print(f"--- Simulating {n_total}-Qubit Evaporation ---")

    # Loop through the evaporation process (emitting one qubit at a time)
    # Stop when the black hole has only one qubit left (or zero if n_total is odd).
    while len(b_qubits) > 1:

        # --- Evaporation Step ---

        # 1. Qubit to be emitted (the "edge" of B)
        qubit_to_emit = b_qubits.pop(0) # Remove from B

        # 2. Apply a more complex unitary operation across the remaining B qubits
        # to model entanglement/scrambling within the black hole before emission
        scrambler_qc = get_scrambling_unitary(b_qubits, n_total)
        current_circuit.compose(scrambler_qc, inplace=True)


        # 3. Move the emitted qubit from B to R (conceptually added to R, index tracked in r_qubits)
        r_qubits.append(qubit_to_emit)


        # --- Entanglement Calculation ---

        # 4. Compute the current quantum state vector
        # This simulates the evolution of the total pure state (B+R)
        state_vector = Statevector.from_instruction(current_circuit)

        # 5. Calculate the reduced density matrix (rho_R) for the radiation (R)
        # We trace out the B qubits (the remaining b_qubits indices)
        b_indices = b_qubits
        rho_R = partial_trace(state_vector, b_indices)

        # 6. Compute the von Neumann entropy (S_vN) of the radiation (base=2 for bits)
        s_vn = entropy(rho_R, base=2)
        entanglement_entropies.append(s_vn)

        print(f"Step {len(r_qubits)}: B size = {len(b_qubits)}, R size = {len(r_qubits)}, S_vN = {s_vn:.4f}")

    return entanglement_entropies

def plot_page_curve(data, n_total):
    """
    Plots the simulated Page Curve and the theoretical S_min bound.
    """

    num_steps = len(data)

    # 1. X-Axis: Number of qubits in the radiation (N_R)
    r_qubit_count = list(range(1, num_steps + 1))

    # 2. Theoretical Page Curve Limit (S_min)
    # The maximum entanglement for a pure state is bounded by the entropy
    # of the smaller subsystem, which is approximated by the qubit count: min(N_R, N_B)
    theoretical_limit = []
    for n_r in r_qubit_count:
        n_b = n_total - n_r
        # The ideal Page Curve is approximated by min(N_R, N_B)
        # for a maximally chaotic (scrambled) process.
        S_min = min(n_r, n_b)
        theoretical_limit.append(S_min)

    # 3. Plotting
    plt.figure(figsize=(10, 6))

    # Simulated Entanglement Entropy (S_vN)
    plt.plot(r_qubit_count, data, label='Simulated Entanglement Entropy $S_{\\text{vN}}(R)$',
             marker='o', linestyle='-', color='blue')

    # Theoretical Bound
    plt.plot(r_qubit_count, theoretical_limit, label='Theoretical Max Entanglement $\\min(N_R, N_B)$',
             linestyle='--', color='red')

    # Page Time Marker
    page_time = n_total / 2
    plt.axvline(x=page_time, color='green', linestyle=':',
                label=f'Page Time ({page_time} qubits)')

    # Aesthetics
    plt.title(f'Simulated Page Curve for a {n_total}-Qubit Black Hole')
    plt.xlabel('Number of Qubits in Radiation Subsystem ($N_R$)')
    plt.ylabel('Entanglement Entropy ($S_{\\text{vN}}$ in bits)')
    plt.xticks(list(range(1, num_steps + 1)) + [page_time]) # Update xticks to include all steps
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.show()
    #

# --- Execution ---
initial_circuit = initialize_system(N_QUBITS)
page_curve_data = simulate_evaporation(initial_circuit, N_QUBITS)
plot_page_curve(page_curve_data, N_QUBITS)